<a href="https://colab.research.google.com/github/lehai0609/KagglePlayground/blob/main/Playground_S5E7_Improving_0_973279.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Overview

In this notebook, I'm doing EDA for the dataset and perform a baseline model based on EDA analysis.
It's a very standardized process, just like the dataset provided in this month's challenge

# I. IMPORT DATA

## 1. Import data

In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost as xgb

# Define the base path to the dataset folder in Google Drive
base_path = '/content/drive/MyDrive/Kaggle/Playground S5E7'

# Define the paths to the individual CSV files
train_path = f'{base_path}/train.csv'
test_path = f'{base_path}/test.csv'
sample_submission_path = f'{base_path}/sample_submission.csv'
original_df_1_path = f'{base_path}/original_df_1.csv'
original_df_2_path = f'{base_path}/original_df_2.csv'

# Read the CSV files into pandas DataFrames
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
sample_submission_df = pd.read_csv(sample_submission_path)
original_df_1 = pd.read_csv(original_df_1_path)
original_df_2 = pd.read_csv(original_df_2_path)

# Display the head of each DataFrame to verify the import
print("Train DataFrame Head:")
display(train_df.head())

print("\nTest DataFrame Head:")
display(test_df.head())

print("\nSample Submission DataFrame Head:")
display(sample_submission_df.head())

print("\nOriginal DataFrame 1 Head:")
display(original_df_1.head())

print("\nOriginal DataFrame 2 Head:")
display(original_df_2.head())

Train DataFrame Head:


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert



Test DataFrame Head:


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,18524,3.0,No,7.0,4.0,No,6.0,NaN
1,18525,NaN,Yes,0.0,0.0,Yes,5.0,1.0
2,18526,3.0,No,5.0,6.0,No,15.0,9.0
3,18527,3.0,No,4.0,4.0,No,5.0,6.0
4,18528,9.0,Yes,1.0,2.0,Yes,1.0,1.0



Sample Submission DataFrame Head:


,id,Personality
0,18524,Extrovert
1,18525,Extrovert
2,18526,Extrovert
3,18527,Extrovert
4,18528,Extrovert



Original DataFrame 1 Head:


,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert
1,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert
2,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert
3,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert
4,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert



Original DataFrame 2 Head:


,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert
1,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert
2,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert
3,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert
4,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert


In [ ]:
# Check the shape of the training and test datasets
print("Shape of train_df:", train_df.shape)
print("Shape of test_df:", test_df.shape)

# Look at basic info about data types and memory usage for the training dataset
print("\nInfo of train_df:")
display(train_df.info())

# Look at basic info about data types and memory usage for the test dataset
print("\nInfo of test_df:")
display(test_df.info())

Shape of train_df: (18524, 9)
Shape of test_df: (6175, 8)

Info of train_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18524 entries, 0 to 18523
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         18524 non-null  int64  
 1   Time_spent_Alone           17334 non-null  float64
 2   Stage_fear                 16631 non-null  object 
 3   Social_event_attendance    17344 non-null  float64
 4   Going_outside              17058 non-null  float64
 5   Drained_after_socializing  17375 non-null  object 
 6   Friends_circle_size        17470 non-null  float64
 7   Post_frequency             17260 non-null  float64
 8   Personality                18524 non-null  object 
dtypes: float64(5), int64(1), object(3)
memory usage: 1.3+ MB


None


Info of test_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6175 entries, 0 to 6174
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         6175 non-null   int64  
 1   Time_spent_Alone           5750 non-null   float64
 2   Stage_fear                 5577 non-null   object 
 3   Social_event_attendance    5778 non-null   float64
 4   Going_outside              5709 non-null   float64
 5   Drained_after_socializing  5743 non-null   object 
 6   Friends_circle_size        5825 non-null   float64
 7   Post_frequency             5767 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 386.1+ KB


None

**OBSERVATION:**

This's a standard 2 class classification challenge.

The target is Personality (Introvert/Extrovert). There are 7 features, 5 numerical features & 2 categorical features ['Stage_fear', 'Drained_after_socializing']

There seams to be some amount of missing values in both train & test dataset, we would need to take care of.

In [ ]:
numerical_features = ['Time_spent_Alone',
                      'Social_event_attendance',
                      'Going_outside',
                      'Friends_circle_size',
                      'Post_frequency']

for col in numerical_features:
    print(f"\nUnique values and counts for '{col}':")
    display(train_df[col].value_counts().sort_index().to_frame(name='Count'))


Unique values and counts for 'Time_spent_Alone':


,Count
Time_spent_Alone,
0.0,3139
1.0,2973
2.0,3039
3.0,3081
4.0,1079
5.0,633
6.0,574
7.0,581
8.0,582



Unique values and counts for 'Social_event_attendance':


,Count
Social_event_attendance,
0.0,1055
1.0,978
2.0,1063
3.0,1703
4.0,2073
5.0,1985
6.0,1984
7.0,2031
8.0,1945



Unique values and counts for 'Going_outside':


,Count
Going_outside,
0.0,1324
1.0,1135
2.0,1279
3.0,2822
4.0,2703
5.0,2923
6.0,2702
7.0,2170



Unique values and counts for 'Friends_circle_size':


,Count
Friends_circle_size,
0.0,467
1.0,753
2.0,770
3.0,866
4.0,1317
5.0,1669
6.0,1146
7.0,1124
8.0,1281



Unique values and counts for 'Post_frequency':


,Count
Post_frequency,
0.0,1272
1.0,1214
2.0,1291
3.0,2258
4.0,1688
5.0,1788
6.0,1767
7.0,1827
8.0,1734


## 2. Missing values

In [ ]:
# Calculate missing values for train_df
train_missing = train_df.isnull().sum()
train_missing_percentage = (train_missing / len(train_df)) * 100

# Calculate missing values for test_df
test_missing = test_df.isnull().sum()
test_missing_percentage = (test_missing / len(test_df)) * 100

# Create a DataFrame to summarize missing values
missing_summary = pd.DataFrame({
    'Train Missing Count': train_missing,
    'Train Missing Percentage (%)': train_missing_percentage,
    'Test Missing Count': test_missing,
    'Test Missing Percentage (%)': test_missing_percentage
})

# Display the missing values summary table
display(missing_summary)

,Train Missing Count,Train Missing Percentage (%),Test Missing Count,Test Missing Percentage (%)
Drained_after_socializing,1149,6.202764,432.0,6.995951
Friends_circle_size,1054,5.689916,350.0,5.668016
Going_outside,1466,7.914057,466.0,7.546559
Personality,0,0.000000,NaN,NaN
Post_frequency,1264,6.823580,408.0,6.607287
Social_event_attendance,1180,6.370114,397.0,6.429150
Stage_fear,1893,10.219175,598.0,9.684211
Time_spent_Alone,1190,6.424098,425.0,6.882591
id,0,0.000000,0.0,0.000000


**OBSERVATION:**

All features have a missing rate at above 5%. Stage_fear has 10.219% of missing at train_df.



In [ ]:
# Calculate missing values for original_df_1
original_df_1_missing = original_df_1.isnull().sum()
original_df_1_missing_percentage = (original_df_1_missing / len(original_df_1)) * 100

# Calculate missing values for original_df_2
original_df_2_missing = original_df_2.isnull().sum()
original_df_2_missing_percentage = (original_df_2_missing / len(original_df_2)) * 100

# Create a DataFrame to summarize missing values
original_missing_summary = pd.DataFrame({
    'Original DF 1 Missing Count': original_df_1_missing,
    'Original DF 1 Missing Percentage (%)': original_df_1_missing_percentage,
    'Original DF 2 Missing Count': original_df_2_missing,
    'Original DF 2 Missing Percentage (%)': original_df_2_missing_percentage
})

# Display the missing values summary table
display(original_missing_summary)

,Original DF 1 Missing Count,Original DF 1 Missing Percentage (%),Original DF 2 Missing Count,Original DF 2 Missing Percentage (%)
Time_spent_Alone,0,0.0,63,2.172414
Stage_fear,0,0.0,73,2.517241
Social_event_attendance,0,0.0,62,2.137931
Going_outside,0,0.0,66,2.275862
Drained_after_socializing,0,0.0,52,1.793103
Friends_circle_size,0,0.0,77,2.655172
Post_frequency,0,0.0,65,2.241379
Personality,0,0.0,0,0.000000


**OBSERVATION:**

Same with original dataset, the 1st part has 0 missing values but the 2nd part has about 2% of missing values.

**SUGGESTION:**

So we need to impute missing values for both train_df, test_df & original_df_2.

For numerical features, my prefer approach is IterativeImputer (MICE - Multiple Imputation by Chained Equations). The imputer will predict missing values from other values of the observation by fitting a regression model. By this, it will preserve the relationship among variables.

For categorical features, I'll simply mode impute to missing values.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
import numpy as np

# Identify numerical and categorical columns (excluding the target 'Personality' from imputation)
numerical_cols = train_df.select_dtypes(include=np.number).columns.tolist()
# Remove 'id' from numerical columns as it's an identifier and not a feature for imputation
numerical_cols.remove('id')
categorical_cols = train_df.select_dtypes(include='object').columns.tolist()
# Remove 'Personality' from categorical columns as it's the target variable
if 'Personality' in categorical_cols:
    categorical_cols.remove('Personality')
display(numerical_cols)
display(categorical_cols)

['Time_spent_Alone',
 'Social_event_attendance',
 'Going_outside',
 'Friends_circle_size',
 'Post_frequency']

['Stage_fear', 'Drained_after_socializing']

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

def impute_datasets(train_df, test_df, original_df_2, numerical_cols, categorical_cols):
    # Initiate MICE for numerical features and mode imputer for categorical features
    iterative_imputer = IterativeImputer(max_iter=10, random_state=0, min_value=0)
    mode_imputer = SimpleImputer(strategy='most_frequent')

    train_df[numerical_cols] = iterative_imputer.fit_transform(train_df[numerical_cols])
    train_df[categorical_cols] = mode_imputer.fit_transform(train_df[categorical_cols])

    test_df[numerical_cols] = iterative_imputer.transform(test_df[numerical_cols])
    test_df[categorical_cols] = mode_imputer.transform(test_df[categorical_cols])

    original_df_2[numerical_cols] = iterative_imputer.transform(original_df_2[numerical_cols])
    original_df_2[categorical_cols] = mode_imputer.transform(original_df_2[categorical_cols])

    return train_df, test_df, original_df_2

train_df, test_df, original_df_2 = impute_datasets(train_df, test_df, original_df_2, numerical_cols, categorical_cols)

# Verify that there are no more missing values in the relevant columns
print("Missing values after imputation (train_df):")
display(train_df[numerical_cols + categorical_cols].isnull().sum())

print("\nMissing values after imputation (test_df):")
display(test_df[numerical_cols + categorical_cols].isnull().sum())

print("\nMissing values after imputation (original_df_2):")
display(original_df_2[numerical_cols + categorical_cols].isnull().sum())

Missing values after imputation (train_df):


,0
Time_spent_Alone,0
Social_event_attendance,0
Going_outside,0
Friends_circle_size,0
Post_frequency,0
Stage_fear,0
Drained_after_socializing,0



Missing values after imputation (test_df):


,0
Time_spent_Alone,0
Social_event_attendance,0
Going_outside,0
Friends_circle_size,0
Post_frequency,0
Stage_fear,0
Drained_after_socializing,0



Missing values after imputation (original_df_2):


,0
Time_spent_Alone,0
Social_event_attendance,0
Going_outside,0
Friends_circle_size,0
Post_frequency,0
Stage_fear,0
Drained_after_socializing,0


# II. BASELINE MODELLING

I am building a simple XGBoost pipeline for classification while:
- Using class weight for imbalance target variables: 0.72/0.28 (extrovert/introvert)
- Stratified K-fold
- Label encoder for categorical features (Stage_fear & Drained_after_socializing)

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import StratifiedKFold, cross_validate
# from sklearn.preprocessing import LabelEncoder
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# import xgboost as xgb

# def create_xgboost_pipeline(df, target_col='personality'):
#     """
#     Simple XGBoost pipeline for personality classification

#     Parameters:
#     - df: DataFrame with features and target
#     - target_col: name of target column

#     Returns:
#     - pipeline: fitted sklearn pipeline
#     - cv_results: cross-validation results
#     """

#     # Define feature columns
#     numerical_features = ['Time_spent_Alone', 'Social_event_attendance',
#                          'Going_outside', 'Friends_circle_size', 'Post_frequency']
#     categorical_features = ['Stage_fear', 'Drained_after_socializing']

#     # Prepare features
#     X = df[numerical_features + categorical_features].copy()
#     y = df[target_col]

#     # Encode binary categorical features (Yes/No -> 1/0)
#     le_stage = LabelEncoder()
#     le_drained = LabelEncoder()

#     X['Stage_fear'] = le_stage.fit_transform(X['Stage_fear'])
#     X['Drained_after_socializing'] = le_drained.fit_transform(X['Drained_after_socializing'])

#     # Encode target
#     le_target = LabelEncoder()
#     y_encoded = le_target.fit_transform(y)

#     # Calculate class weights for imbalanced data (0.28 introvert, 0.72 extrovert)
#     # Assuming 0=introvert, 1=extrovert after encoding
#     scale_pos_weight = 0.72 / 0.28  # ≈ 2.57

#     # Create XGBoost model with class balancing
#     xgb_model = xgb.XGBClassifier(
#         random_state=42,
#         scale_pos_weight=scale_pos_weight,
#         n_estimators=100,
#         max_depth=6,
#         learning_rate=0.1
#     )

#     # Create simple pipeline (no preprocessing needed since we already encoded)
#     pipeline = Pipeline([
#         ('classifier', xgb_model)
#     ])

#     # Stratified K-fold cross-validation
#     cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#     # Scoring metrics
#     scoring = {
#         'accuracy': 'accuracy',
#         'precision': 'precision',
#         'recall': 'recall',
#         'f1': 'f1',
#         'roc_auc': 'roc_auc'
#     }

#     # Perform cross-validation
#     cv_results = cross_validate(
#         pipeline, X, y_encoded,
#         cv=cv,
#         scoring=scoring,
#         return_train_score=False,
#         n_jobs=-1
#     )

#     # Fit final pipeline
#     pipeline.fit(X, y_encoded)

#     return pipeline, cv_results

# def print_cv_results(cv_results):
#     """Print cross-validation results"""
#     print("Cross-Validation Results:")
#     print("-" * 40)

#     for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
#         scores = cv_results[f'test_{metric}']
#         print(f"{metric.upper():>10}: {scores.mean():.4f} ± {scores.std():.4f}")

# Run pipeline
# pipeline, cv_results = create_xgboost_pipeline(train_df, target_col='Personality')

# Print results
# print_cv_results(cv_results)


In [ ]:
# Prepare the test data

# X_test = test_df[numerical_features + categorical_features].copy()
# # We need the fitted LabelEncoders from the training step
# # Since they were not returned by the create_xgboost_pipeline function,
# # we will quickly refit them on the training data to ensure consistency
# le_stage = LabelEncoder()
# le_drained = LabelEncoder()

# # Fit on the training data's categorical columns
# le_stage.fit(train_df['Stage_fear'])
# le_drained.fit(train_df['Drained_after_socializing'])

# # Transform the test data's categorical columns
# X_test['Stage_fear'] = le_stage.transform(X_test['Stage_fear'])
# X_test['Drained_after_socializing'] = le_drained.transform(X_test['Drained_after_socializing'])

# # Make predictions on the test data using the trained pipeline
# test_predictions_encoded = pipeline.predict(X_test)

# # Decode the predictions back to original labels ('Introvert', 'Extrovert')
# # We need the fitted LabelEncoder for the target variable from the training step
# # Similar to the feature encoders, we will refit the target encoder
# le_target = LabelEncoder()
# le_target.fit(train_df['Personality'])

# test_predictions = le_target.inverse_transform(test_predictions_encoded)

# # Create the submission DataFrame
# submission_df = pd.DataFrame({
#     'id': test_df['id'],
#     'Personality': test_predictions
# })

# # Display the head of the submission DataFrame
# print("\nSubmission DataFrame Head:")
# display(submission_df.head())

# # Save the submission DataFrame to a CSV file
# submission_df.to_csv('submission.csv', index=False)

# print("\nSubmission file 'submission.csv' created successfully.")

# III. TARGET ENCODED APPROACH